In [1]:
# Imports
import numpy as np
from matplotlib import pyplot as plt
from scipy import signal as sp

# Load the raw collection
s = np.fromfile(open("rawData"), dtype=np.complex64)

# Notes from Matthew
Fs = 1e6
Fc = 433946924

# Setup a time axis
t = np.arange(s.size) / Fs

In [2]:
def filterSignal(s, timeframe, Fs):
    # Low =-pass filter the data
    h = sp.firwin(numtaps=101, cutoff=1, nyq=Fs/2)
    s_filtered = np.abs(sp.lfilter(h, 1.0, s[timeframe]))
    return s_filtered

def getStartTime(s):
    # Find the start of the first data chunk
    start = 0
    for i, item in enumerate(s):
        if(item > 0.05):
            start = i
            break
    if(start == 0):
        # no signal found
        return -1
    else:
        return start

def signal_to_data(s_filtered):
    # First find the mean amplitude of the filtered data signal
    avg = sum(s_filtered) / s_filtered.size
    # Anything that is below the mean is 0, and anything above is 1
    scaled = s_filtered.copy()
    for i, item in enumerate(scaled):
        if item < avg:
            scaled[i] = 0
        else:
            scaled[i] = 1
    # Find the duration of each peak and place them into an array
    durations = []
    bits = []
    prevVal = scaled[0]
    duration = 0
    for i in range(scaled.size):
        if(scaled[i] == prevVal):
            duration = duration + 1
        else:
            durations.append(duration)
            if(duration > 580):
                bits.append(bool(prevVal))
                bits.append(bool(prevVal))
                bits.append(bool(prevVal))
            elif(duration > 380):
                bits.append(bool(prevVal))
                bits.append(bool(prevVal))
            else:
                bits.append(bool(prevVal))
            duration = 0 # reset the duration "timer"
        prevVal = scaled[i]

    # find the max and min durations
    maxDuration = np.max(durations)
    minDuration = np.min(durations)
    print('Min Duration:', minDuration)
    print('Max Duration:', maxDuration)
    print('Total Pulses:', len(durations))
    print('Number of Bits:', len(bits))
    
    byteStr = ""
    for b in bits:
        byteStr += str(int(b))
    return byteStr

def compare_byteStrings(b1, b2, b3):
    return b1 == b2 and b2 == b3

In [3]:
# plt.figure(figsize=(12,5))
# plt.plot(t, np.abs(s))
# plt.xlabel('Time')
# plt.ylabel('$|s|$')
# plt.grid()
# plt.show()

In [4]:
# plt.figure(figsize=(12,5))
# plt.plot(t, np.abs(s))
# plt.xlabel('Time')
# plt.ylabel('$|s|$')
# plt.xlim(6, 6.25)
# plt.grid()
# plt.show()

In [5]:
# plt.figure(figsize=(12,5))
# plt.plot(t, np.abs(s))
# plt.xlabel('Time')
# plt.ylabel('$|s|$')
# plt.xlim(6.0955, 6.1375)
# plt.grid()
# plt.show()

In [6]:
# # Plot just the real
# plt.figure(figsize=(12,5))
# plt.plot(t, np.real(s))
# plt.xlabel('Time')
# plt.ylabel('$|s|$')
# plt.xlim(6.012, 6.054)
# plt.grid()
# plt.show()

In [7]:
# # Plot just the imaginary
# plt.figure(figsize=(12,5))
# plt.plot(t, np.imag(s))
# plt.xlabel('Time')
# plt.ylabel('$|s|$')
# plt.xlim(6.012, 6.054)
# plt.grid()
# plt.show()

In [8]:
# timeframe1 = np.where(np.logical_and(t>=6.012, t<=6.054))[0]
# timeframe2 = np.where(np.logical_and(t>=6.05375, t<=6.09575))[0]
# timeframe3 = np.where(np.logical_and(t>=6.0955, t<=6.1375))[0]
startTime = (getStartTime(s)/Fs) - 0.00125
print(startTime)
timeframe1 = np.where(np.logical_and(t>=startTime, t<=startTime+0.042))[0]
timeframe2 = np.where(np.logical_and(t>=startTime+0.042, t<=startTime+0.084))[0]
timeframe3 = np.where(np.logical_and(t>=startTime+0.084, t<=startTime+0.126))[0]

6.0119430000000005


In [9]:
# # Make an IQ plot
# plt.figure(figsize=(5,5))
# plt.plot(np.real(s[timeframe1]), np.imag(s[timeframe1]), '.', ms=1)
# plt.xlabel('Real')
# plt.ylabel('Imaginary')
# plt.xlim(-0.3,0.3)
# plt.ylim(-0.3,0.3)
# plt.gca().set_aspect('equal')
# plt.grid()
# plt.show()

In [10]:
# plt.specgram(s[timeframe1], Fs=Fs)
# plt.ylim(-12500,12500)
# plt.xlabel('Time')
# plt.ylabel('Frequency')
# plt.show()

In [11]:
s_filtered1 = filterSignal(s, timeframe1, Fs)
b1 = signal_to_data(s_filtered1)
print(b1)

Min Duration: 203
Max Duration: 1297
Total Pulses: 122
Number of Bits: 196
0001110001110001110001110001101101001001001001001101001101101101101101101101001101001001001101001001101001001101001001001001001001001001101001001101001101001001101001001001001101101001001101001101


In [12]:
s_filtered2 = filterSignal(s, timeframe2, Fs)
b2 = signal_to_data(s_filtered2)
print(b2)

Min Duration: 203
Max Duration: 1046
Total Pulses: 122
Number of Bits: 196
0001110001110001110001110001101101001001001001001101001101101101101101101101001101001001001101001001101001001101001001001001001001001001101001001101001101001001101001001001001101101001001101001101


In [13]:
s_filtered3 = filterSignal(s, timeframe3, Fs)
b3 = signal_to_data(s_filtered3)
print(b3)

Min Duration: 203
Max Duration: 794
Total Pulses: 122
Number of Bits: 196
0001110001110001110001110001101101001001001001001101001101101101101101101101001101001001001101001001101001001101001001001001001001001001101001001101001101001001101001001001001101101001001101001101


In [14]:
print(compare_byteStrings(b1, b2, b3))

True
